<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2020ccap/2020_0521ccap_softmax_with_temparature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ccap 勉強会資料
- date: 2020-0521
- author: 浅川伸一
- title: 注意に使用されるソフトマックス関数は，温度を付けることができる



---

- 鈴木さんに解説していただいたとおり，注意とはソフトマックス関数のことです。
- そして，ソフトマックス関数とは，複数の選択肢の中から，最大の値を与える選択肢の値を強調し，
- かつ，全体の総和が 1 となるような変換を指します。
- 従って，どのような値であれ，実数値 スカラ であれば，複数の選択肢の選択確率を与えます。
- また，よく知られているとおり，選択する確率を $p$ とすれば，選択しない確率は $1-p$ ですから，
- この 2 つの値でソフトマックス関数を行った場合をシグモイド関数 (the logistic sigmoid function) と呼びます。
$$
\frac{e^p}{e^p + e^{1-p}} = \frac{1}{1+e^{\frac{1-p}{p}}}
= \frac{1}{1+e^{-\frac{p}{1-p}}}
$$
上式で，選択確率 $p$ と非選択確率 $(1-p)$ の比 $\displaystyle\frac{p}{1-p}$ を $x$ とおくと
よく知られたシグモイド関数の定義式を得ます。

$$
f(x) =\sigma(x) = \left[1+e^{-x}\right]^{-1}
$$

すなわち，ニューラルネットワークで以前頻繁に用いられていたシグモイド関数は，ソフトマックス関数の特殊な場合であると見なすことが可能です。

- 加えて，LSTM や GRU に出てくる **ゲート** とは，シグモイド関数ですから，この値を信用するか，あるいは信用しないかを，表す比を使って，情報を流すか，流さないかを制御するカラクリであると考えることができます。

- たとえば，絵画呼称課題において，脳内に想起された回答候補を選択する確率が，その内部刺激表現の強度のみならず，
この温度にも影響を受けると考えれば，故障障害の記述に洞察を与えることができるだろうと考えます。
- そして，この温度パラメータに相当する，神経対応物はセロトニンやサイトカインなどの神経修飾物質の脳内濃度ではないかと考えることができます。（浅川の妄想です）

In [0]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.misc import # 古い python のバージョンでは logsumexp の定義されている場所が異なります。
from scipy.special import logsumexp

In [0]:
def softmax_wT(x, T=1., axis=None):
    """
    Softmax function

    The softmax function transforms each element of a collection by
    computing the exponential of each element divided by the sum of the
    exponentials of all the elements.  That is, if `x` is a one-dimensional
    numpy array::

        softmax(x) = np.exp(x)/sum(np.exp(x))

    Parameters
    ----------
    x : array_like
        Input array.
    axis : int or tuple of ints, optional
        Axis to compute values along. Default is None and softmax will be
        computed over the entire array `x`.

    Returns
    -------
    s : ndarray
        An array the same shape as `x`. The result will sum to 1 along the
        specified axis.
    """

    return np.exp(x/T - logsumexp(x/T, axis=axis, keepdims=True))

In [0]:
np.random.seed(3)  # 乱数生成器の種を設定

N = 3  # 生成する乱数の個数
raw_data = np.random.rand(N)  
print(raw_data, softmax_wT(raw_data), softmax_wT(raw_data, T=10))

In [0]:
width = 0.3

plt.figure(figsize=(10,8))
plt.bar(x - width, softmax_wT(raw_data, T=0.1), width, label='softmax T=0.1')
plt.bar(x,         softmax_wT(raw_data, T=1), width, label='softmax T=1')
plt.bar(x + width, softmax_wT(raw_data, T=30), width, label='softmax T=30')
plt.legend()
plt.show()

# LSTM の生物学的対応物

以下の画像は
<http://kybele.psych.cornell.edu/~edelman/Psych-2140/week-2-2.html> より。
ウミウシのエラ引っ込め反応時に，ニューロンへの入力信号ではなく，入力信号を修飾する結合が存在する。

<div align="center">
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/C87-fig2_24.jpg"><br/>
</div>

<div align="center">
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/shunting-inhibition.jpg" style="width:39%"><br />
</center>

- source: http://kybele.psych.cornell.edu/~edelman/Psych-2140/week-2-2.html




# LSTM のゲートによる系列制御

<div align='center'>
<img src="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2020-0521RNN_gating_roles.svg">
</div>

上手のような中間層の再帰結合があれば，$1\rightarrow2\rightarrow3$ のような系列が表現できる。
逆に，行列 $W$ の対角要素は $-1$ になっている。
これは，同じことを 2 度繰り返さない。
復帰抑制 inhibition of return である。<br/>
<font color="teal" size="+2">この値が，障害を受けると保続 preservation が起きると予想される。</font>


---

## 意味埋め込み空間の構造化

あきらかな蛇足だが，意味空間を，関連ベクトルで構成される部分空間と考えると，
意味概念は，各構成要素ベクトルで張られる部分空間であると見なすことができる，
そうすると $s(\text{男})$ は男空間であるし，$I-s(\text{男})$ は男の直交補空間である。
**王** 概念から **男** を引くとは $s(\text{王})$ を 男の直交補空間への射影であること捉えることが可能である。
そのようにしてできた空間へ $s(\text{女})$ 空間との合成空間では，**女王** ベクトルが最も代表的
最大固有値を与えるベクトルとなると予想できる。

同じことを考えると，意味特徴分析(例えば 高倉，大槻，宇野，cnps2018)は，各意味特徴空間への射影行列と
みなしうる。
このように考えれば，<br/>
<font color="teal" size="+2">平坦な単語埋め込みモデルによる空間は，SFA の構造を内包しているとみなせるのではないか</font>



In [0]:
import IPython.display
IPython.display.Image(url="https://raw.githubusercontent.com/ShinAsakawa/ShinAsakawa.github.io/master/assets/2018Efstratiadou_SFA_fig1.svg")
